# Um projeto de *machine learning*

Vamos construir um exemplo de projeto de *machine learning* nestas primeiras aulas. Para isso vamos usar um *dataset* de preços de carros usados.

## Obtendo os dados

O *dataset* vem do *site* 'Kaggle' (https://www.kaggle.com/). Sempre que possível é bom automatizar o processo de *download*, descompactação e leitura dos dados.

In [1]:
import zipfile
from pathlib import Path

import pandas as pd
import requests

DATA_DIR = Path.cwd().resolve().parents[2] / 'datasets' / 'car_price'
_CAR_DATASET_URL = 'https://www.kaggle.com/api/v1/datasets/download/asinow/car-price-dataset'
_TIMEOUT = 10
_COMPRESSED_CAR_DATASET_FILENAME = 'car_price_dataset.zip'
_CAR_DATASET_FILENAME = 'car_price_dataset.csv'

def _fetch_car_dataset(raw_dataset_path: Path, data_dir: Path) -> None:
    '''Fetches the car dataset from Kaggle and saves it to the data_dir.
    '''
    data_dir.mkdir(parents=True, exist_ok=True)
    response = requests.get(_CAR_DATASET_URL, timeout=_TIMEOUT)
    response.raise_for_status()
    with open(raw_dataset_path, 'wb') as f:
        f.write(response.content)


def _unpack_car_dataset(raw_dataset_path: Path, data_dir: Path) -> None:
    '''Unpacks the car dataset from the data_dir.
    '''
    with zipfile.ZipFile(raw_dataset_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)


def _fetch_and_unpack_car_dataset(data_dir: Path) -> None:
    '''Fetches and unpacks the car dataset from Kaggle.
    '''
    raw_dataset_path = data_dir / _COMPRESSED_CAR_DATASET_FILENAME
    _fetch_car_dataset(raw_dataset_path, data_dir)
    _unpack_car_dataset(raw_dataset_path, data_dir)


def load_car_dataset(data_dir: Path) -> None:
    '''Loads the car dataset from the data_dir.
    '''
    dataset_path = data_dir / _CAR_DATASET_FILENAME
    if not dataset_path.exists():
        _fetch_and_unpack_car_dataset(data_dir)
    dataset = pd.read_csv(dataset_path)
    return dataset

In [2]:
dataset = load_car_dataset(DATA_DIR)

Verifique que o código funcionou:

In [3]:
dataset.head()

,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Kia,Rio,2020,4.2,Diesel,Manual,289944,3,5,8501
1,Chevrolet,Malibu,2012,2.0,Hybrid,Automatic,5356,2,3,12092
2,Mercedes,GLA,2020,4.2,Diesel,Automatic,231440,4,2,11171
3,Audi,Q5,2023,2.0,Electric,Manual,160971,2,1,11780
4,Volkswagen,Golf,2003,2.6,Hybrid,Semi-Automatic,286618,3,3,2867


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         10000 non-null  object 
 1   Model         10000 non-null  object 
 2   Year          10000 non-null  int64  
 3   Engine_Size   10000 non-null  float64
 4   Fuel_Type     10000 non-null  object 
 5   Transmission  10000 non-null  object 
 6   Mileage       10000 non-null  int64  
 7   Doors         10000 non-null  int64  
 8   Owner_Count   10000 non-null  int64  
 9   Price         10000 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 781.4+ KB


Parece que deu tudo certo com este dataset.

---

***Atividade***

Verifique se a criação dos arquivos de dados realmente ocorreu.

---

Vamos agora mover o código de leitura dos dados para um pacote Python, de modo a tornar o código disponível para todos.

## Criando um pacote de código Python

Códigos Python são organizados em *scripts*, *módulos* e *pacotes*. Além desses tipos de código, temos também os *notebooks*, que são interativos.

### Scripts

***Scripts*** são arquivos que contêm código Python e são executados diretamente. São arquivos com extensão `.py` feitos para serem executados pelo interpretador Python na linha de comando. Por exemplo:

- Crie um diretório `meu_projeto`. Nele vamos criar um arquivo `meu_script.py`. A estrutura de diretórios ficou assim:

    > ```text
    > meu_projeto/
    > └── meu_script.py
    > ```

- Eis um *script* Python (`meu_script.py`):

    > ```Python
    > '''Imprime uma saudação colorida na tela.
    > '''
    > import colorama
    > 
    > 
    > def main() -> None:
    >     '''Função principal do script.
    >     '''
    >     # Prepara a Colorama para funcionar corretamente,
    >     # especialmente em ambientes Windows.
    >     colorama.init()
    > 
    >     styling = colorama.Fore.RED + colorama.Back.YELLOW + colorama.Style.BRIGHT
    >     reset_styling = colorama.Style.RESET_ALL
    > 
    >     print(styling + 'Hello, World!' + reset_styling)
    > 
    >     colorama.deinit()
    > 
    > 
    > if __name__ == '__main__':
    >     main()
    > ```

- E podemos executar esse *script* na linha de comando:

    > ```bash
    > cd meu_projeto
    > python meu_script.py
    > ```

---

***Atividade***

Escreva e rode os códigos acima para verificar se funcionam.

---

### Módulos

**Módulos** são arquivos que contêm código Python e são importados por outros módulos ou scripts. São arquivos também com extensão `.py` mas cujo propósito é ser *importado* em outros códigos, veja um exemplo:

- No diretório `meu_projeto` adicione o arquivo `meu_modulo.py` abaixo. A estrutura de diretórios fica assim:

    ```text
    meu_projeto/
    ├── meu_modulo.py
    └── meu_script.py
    ```

- `meu_modulo.py`

    > ```Python
    > '''Módulo de exemplo para demonstrar a criação de pacotes.
    > '''
    > import colorama
    > 
    > 
    > def say_hello(name: str | None = 'world') -> None:
    >     '''Imprime uma saudação colorida na tela.
    > 
    >     Args:
    >         name (str, optional): Nome a ser saudado.
    >     '''
    >     # Prepara a Colorama para funcionar corretamente,
    >     # especialmente em ambientes Windows.
    >     colorama.init()
    > 
    >     styling = colorama.Fore.RED + colorama.Back.YELLOW + colorama.Style.BRIGHT
    >     reset_styling = colorama.Style.RESET_ALL
    > 
    >     print(styling + f'Hello, {name}' + reset_styling)
    > 
    >     colorama.deinit()
    > ```

- E agora modifique o *script* `meu_script.py` para que importe o módulo:

    > ```Python
    > '''Imprime uma saudação colorida na tela.
    > '''
    > import meu_modulo
    > 
    > def main() -> None:
    >     '''Função principal do script.
    >     '''
    >     meu_modulo.say_hello('Insper')
    > 
    > if __name__ == '__main__':
    >     main()
    > ```

---

***Atividade***

Escreva e rode os códigos acima para verificar se funcionam.

---

### Pacotes

**Pacotes** são diretórios que contêm módulos e um arquivo especial chamado `__init__.py` (note o uso de *underscores* duplos em ambos os lados. Em Python, esses *underscores* duplos são chamados de *dunders* - *double underscores*). Eis um exemplo:

- Nossa estrutura de arquivos seria assim:

    > ```text
    > meu_script.py
    > meu_pacote/
    > ├── __init__.py
    > └── meu_modulo.py
    > ```

- `meu_modulo.py` é o módulo acima

- `__init__.py` é um arquivo vazio

- `meu_script.py` é assim:

    > ```Python
    > '''Imprime uma saudação colorida na tela.
    > '''
    > import meu_package.meu_modulo as mm
    > 
    > def main() -> None:
    >     '''Função principal do script.
    >     '''
    >     mm.say_hello('Insper')
    > 
    > if __name__ == '__main__':
    >     main()
    > ```

---

***Atividade***

Escreva e rode os códigos acima para verificar se funcionam.

---

### Notebooks

E você está trabalhando aonde mesmo?

### Pacotes instaláveis

Quando fazemos um *pacote* Python, é importante que ele seja *localizável* pelos outros códigos. Pense em *pacotes* que você importa o tempo todo, como o *Pandas*. Este pacote não está no mesmo diretório que seu *script* ou *notebook*, como foi o caso do exemplo acima. Como ele é localizado então na hora de rodar seu código Python que importa o Pandas?

Existem algumas formas de tornar um código *localizável*:

- Colocar o diretório do seu pacote na variável de ambiente `PYTHONPATH`, ou

- Transformar seu pacote em um pacote *instalável*

Prefiro a segunda opção - ela permite que seu código evolua a ponto de um dia ser instalado diretamente do repositório central de pacotes Python, o PyPI!

Para tornar seu código instalável, você precisa criar um arquivo `pyproject.toml` no seu projeto, veja o exemplo:

- A estrutura de arquivos e diretórios do projeto fica assim:

    > ```text
    > meu_projeto/
    > ├── src/
    > │   └── meu_pacote/
    > │       ├── __init__.py
    > │       └── meu_modulo.py
    > ├── notebooks/
    > │   └── meu_notebook.ipynb
    > ├── scripts/
    > │   └── meu_script.py
    > ├── pyproject.toml
    > └── README.md
    > ```

onde `meu_modulo.py` e `meu_script` tem o conteúdo acima.

- O arquivo `pyproject.toml` armazena informações à respeito do seu projeto Python como um todo, incluindo instruções para instalação dos pacotes, configuração de ferramentas de teste, etc. Para nossos propósitos vamos fazer o arquivo `pyproject.toml` mais simples possível:

    > ```toml
    > [build-system]
    > requires = ["setuptools>=61.0"]
    > build-backend = "setuptools.build_meta"
    > 
    > [project]
    > name = "meu_pacote"
    > description = "Código do projeto Saudação"
    > version = "0.1.0"
    > readme = "README.md"
    > dependencies = ["colorama"]
    > ```

Agora você pode instalar o pacote no seu ambiente Python. 

- Inicie o seu ambiente. Por exemplo `conda activate ml`. É sempre uma péssima idéia instalar pacotes no ambiente global (o `base` do `conda`, ou o Python do sistema).

- Com o ambiente inicializado, mude seu diretório de trabalho no terminal para o diretório onde o arquivo `pyproject.toml` está.

- Agora execute o comando de instalação do seu pacote no seu ambiente Python:

    > ```bash
    > pip install -e .
    > ```

Com isso você está instruindo o comando `pip`, de gerenciamento de pacotes Python, a instalar o seu pacote. O *flag* `-e` significa "editável": você pode fazer alterações à vontade no seu pacote instalado, e essas alterações estarão disponíveis para seus outros códigos.

<hr/>

***Atividade***:

Escreva e rode os códigos acima para ver que funcionam.

<hr/>

***Atividade***:

Crie um notebook `meu_notebook.ipynb` na pasta `notebooks` de `meu_projeto`. Neste *notebook* você deve importar adequadamente o módulo `meu_modulo` do seu pacote `meu_pacote`. Chame a função `say_hello`.


<hr/>





Agora que você domina a criação de pacotes instaláveis Python, vamos criar o projeto "Car Prices".

<hr/>

***Atividade***

Crie a seguinte estrutura de diretórios para o seu projeto:

> ```text
> car_prices/
> ├── src/
> │   └── car_prices/
> │       ├── __init__.py
> │       └── dataset.py
> ├── notebooks
> ├── scripts
> ├── pyproject.toml
> └── README.md
> ```

Agora transfira o código de leitura do *dataset* para o módulo `dataset.py`. Note que temos uma questão acerca da variável `DATA_DIR`, que não deve ser transferida.